# 1.Develop a DCGAN model for CIFAR-10 small object Photograph Dataset, for implement this model-create a generator and discriminator model and then generate the new set of fake images to ensure the GAN outcome. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 dataset
(x_train, _), (_, _) = cifar10.load_data()
x_train = (x_train - 127.5) / 127.5  # Normalize to range [-1, 1]

# Generator Model
def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(4 * 4 * 256, input_dim=latent_dim))
    model.add(layers.Reshape((4, 4, 256)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2DTranspose(3, (4, 4), activation='tanh', padding='same'))
    return model

# Discriminator Model
def build_discriminator(img_shape):
    model = keras.Sequential()
    model.add(layers.Conv2D(64, (3, 3), padding='same', input_shape=img_shape))
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Define GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Set random seed for reproducibility
np.random.seed(1000)

# Build and compile the discriminator
img_shape = (32, 32, 3)
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

# Build and compile the generator
latent_dim = 100
generator = build_generator(latent_dim)
generator.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Build and compile the GAN model
discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Training the GAN
epochs = 10000
batch_size = 64

# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):
    # Train Discriminator
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_imgs = x_train[idx]
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_imgs, valid)
    d_loss_fake = discriminator.train_on_batch(fake_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, valid)

    # Print progress
    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

    # Save generated images at specified intervals
    if epoch % 1000 == 0:
        generated_imgs = generator.predict(np.random.normal(0, 1, (16, latent_dim)))
        generated_imgs = 0.5 * generated_imgs + 0.5  # Rescale to [0, 1]
        fig, axs = plt.subplots(4, 4)
        count = 0
        for i in range(4):
            for j in range(4):
                axs[i, j].imshow(generated_imgs[count, :, :, :])
                axs[i, j].axis('off')
                count += 1
        plt.show()


# 2.Explore the Latent Space when Generating images.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

# Load pre-trained generator model
generator = keras.models.load_model('path_to_your_generator_model')  # Replace with the actual path

# Function to generate images from points in the latent space
def generate_images(generator, latent_dim, n_samples=16):
    # Generate random points in the latent space
    latent_points = np.random.normal(0, 1, (n_samples, latent_dim))

    # Generate images from the latent points
    generated_images = generator.predict(latent_points)

    return generated_images

# Visualize generated images at different points in the latent space
latent_dim = 100
num_samples_per_dimension = 8
total_samples = num_samples_per_dimension ** 2

generated_images = generate_images(generator, latent_dim, total_samples)

# Rescale images to [0, 1]
generated_images = 0.5 * generated_images + 0.5

# Create a grid of generated images
fig, axs = plt.subplots(num_samples_per_dimension, num_samples_per_dimension, figsize=(8, 8))

count = 0
for i in range(num_samples_per_dimension):
    for j in range(num_samples_per_dimension):
        axs[i, j].imshow(generated_images[count, :, :, :])
        axs[i, j].axis('off')
        count += 1

plt.show()


This code generates a grid of images by randomly sampling points from the latent space and passing them through the pre-trained generator. Each row in the grid corresponds to a different dimension in the latent space, and each column corresponds to different values along that dimension.